### Calcoliamo un indice di segregazione per ciascun quartiere e ciascun anno del dataset

In [45]:
import json
dirPath = '/Users/albertocottica/Documents/GitHub/-milano-melting-pot/dati/'
with open (dirPath + 'map1Data_ultracompact.json', 'r') as jsonfile:
    data = json.load(jsonfile)
print(len(data))
print (data[:4])

3520
[{'Anno': '1999', 'IdNil': '1', 'eastas': 71, 'noraf': 102, 'ssaf': 141, 'seas': 309, 'ita': 17602, 'southas': 198, 'soam': 260, 'eu15': 663, 'westas': 124, 'noram': 147, 'eun13': 56, 'otheu': 134, 'ocean': 5, 'other': 5}, {'Anno': '1999', 'IdNil': '2', 'eastas': 105, 'noraf': 94, 'ssaf': 181, 'seas': 251, 'ita': 17523, 'southas': 147, 'soam': 243, 'eu15': 660, 'westas': 38, 'noram': 163, 'eun13': 41, 'otheu': 166, 'ocean': 8, 'other': 3}, {'Anno': '1999', 'IdNil': '3', 'eastas': 0, 'noraf': 0, 'ssaf': 2, 'seas': 6, 'ita': 32, 'southas': 3, 'soam': 1, 'eu15': 8, 'westas': 0, 'noram': 1, 'eun13': 0, 'otheu': 0, 'ocean': 0, 'other': 0}, {'Anno': '1999', 'IdNil': '4', 'eastas': 73, 'noraf': 103, 'ssaf': 141, 'seas': 298, 'ita': 14761, 'southas': 185, 'soam': 253, 'eu15': 421, 'westas': 108, 'noram': 85, 'eun13': 51, 'otheu': 172, 'ocean': 3, 'other': 2}]


Calcoliamo l'indice di segregazione. L'idea è questa: per ciascun anno e per ciascun quartiere devo trovare tutti i record con i numeri di tutte le nazionalità, e metterli in un vettore. A questo punto calcolo l'entropia del vettore, e salvo il risultato in un dizionario.

In [42]:
import math
def segregation_index(nats):
    '''
    (list) => float
    ritorna l'indice di segregazione associato a una popolazione divisa in componenti. 
    Ciascun componente ha una numerosità, che è un elemento di nats.
    '''
    base = len(nats) # the logarithm's base
    pop = sum(nats)
    index = 0
    for nat in nats:
        if nat == 0:
            nat = 0.000000001 # log(0) is undefined and breaks the code
        index += nat/pop * math.log(nat/pop, base)
    return -index

## test
mylist = [1,2,3,4,5,6,7,8,9]
print (segregation_index(mylist))

0.9329227160192092


In [46]:
map3Data = []
for item in data:
    nationalities = []
    for key in item:
        if key != 'Anno' and key != 'IdNil':
            nationalities.append(int(item[key]))
    newitem = {'Anno': item['Anno'], 'IdNil': item['IdNil'], 'SI': segregation_index(nationalities)}
    map3Data.append(newitem)
print(map3Data[:4])

[{'Anno': '1999', 'IdNil': '1', 'SI': 0.2242727988555109}, {'Anno': '1999', 'IdNil': '2', 'SI': 0.21539763514080623}, {'Anno': '1999', 'IdNil': '3', 'SI': 0.48225983966002406}, {'Anno': '1999', 'IdNil': '4', 'SI': 0.23053975335074206}]


Salviamo...

In [49]:
with open (dirPath + 'map3Data.json', 'w') as jsonfile:
    json.dump(map3Data, jsonfile)
print('Done')

Done
